In [ ]:
pip install datasets

In [ ]:
pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import datasets
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd

In [ ]:
# Initialize Spark session
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder.appName("ResourceOptimizedSession1").master("local[*]").config("spark.driver.memory", "16g").config("spark.executor.memory", "16g").config("spark.driver.maxResultSize", "4g").config("spark.executor.heartbeatInterval", "30s").getOrCreate()

In [ ]:
def pricing_sensitivty(year):
  datasets.logging.set_verbosity_error()
  dataset = load_from_disk("/content/drive/MyDrive/MIT805/Merged/Books_{}".format(year))

  df = dataset.to_pandas()
  df['categories'] = df['categories'].apply(lambda x: ', '.join(x) if len(x) > 0 else '').astype(str)

  pyspark_df = spark.createDataFrame(df[['parent_asin', 'rating', 'price', 'timestamp', 'book_title','store', 'categories']])\
                    .withColumn("timestamp", f.date_format(f.from_unixtime(f.col("timestamp") / 1000), "yyyy-MM-dd"))
  pyspark_df = pyspark_df.withColumn('timestamp', f.to_date('timestamp')).withColumn('year-month', f.substring('timestamp', 1, 4))
  grouped_df = pyspark_df.groupby(['parent_asin', 'price', 'year-month', 'book_title','store', 'categories']).mean().withColumnRenamed("avg(rating)", "avg_rating")
  grouped_df.write.csv("/content/drive/MyDrive/MIT805/Results/Pricing_Sensitivity_Analysis/{}/".format(year), header=True, mode="overwrite")
  print("Year {} is done!".format(year))

In [ ]:
for year in range(2010, 2024):
  pricing_sensitivty(year)